##### Copyright 2018 The TensorFlow Authors. [Licensed under the Apache License, Version 2.0](#scrollTo=bPJq2qP2KE3u).

In [ ]:
// #@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
// https://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/swift/tutorials/raw_tensorflow_operators"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/swift/blob/master/docs/site/tutorials/raw_tensorflow_operators.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/swift/blob/master/docs/site/tutorials/raw_tensorflow_operators.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# 다듬어지지 않은 텐서플로우 연산자들

텐서플로우에서 빌드하는 것과, 텐서플로우의 Swift는 API design이 새로운 접근을 할 수 있게 합니다. API들은 확립된 라이브러리들로 부터 세밀하게 조직되어 있고, 새로운 관용구로 결합되어 있습니다. 이것은 모든 텐서플로우 API들이 직접적으로 Swift APIs로 사용되어질 수 있는 것은 아니라는 걸 뜻하고, 또 우리의 API curation이 상당한 시간과 발전 시키기 위한 헌신적인 노력이 필요하다는 것을 의미합니다. 만약 당신의 가장 좋아하는 텐서플로우 연산자가 Swift에서 사용 가능 하지 않는다고 해도 걱정하지 마세요. – 텐서플로우 Swift 라이브러리는 당신에게 Raw namespace 아래에서 대부분의 텐서플로우 연산자에 접근할 수 있도록 합니다..


`TensorFlow`를 시작하기 위해 불러오세요.

In [ ]:
import TensorFlow

## 다듬어지지 않은 연산자 호출하기

코드 완성기능을 통해 `Raw` namespace에서 당신이 필요한 함수를 찾으세요.

In [ ]:
Raw.mul(Tensor([2.0, 3.0]), Tensor([5.0, 6.0]))

## 새로운 곱셈 연산자를 정의하기

곱셈은 이미 `Tensor` 에서 `*` 연산자로 사용이 가능하지만, `.*`라는 새로운 이름의 연산자로 이용 가능 할 수 있도록 만들고 싶다고 가정해봅시다. Swift는 당신이 소급적으로 메소드를 추가할 수 있도록 하거나 `extension` 선언을 사용하는 타입들이 존재할 가능성을 계산할 수 있게 해줍니다.

지금, `.*`를 extension을 선언하여 `Tensor`에 추가해봅시다. 그리고 tensor의 `Scalar` 타입이 [`Numeric`](https://developer.apple.com/documentation/swift/numeric)을 따를 때 이용가능 하도록 만드세요.

In [ ]:
infix operator .* : MultiplicationPrecedence

extension Tensor where Scalar: Numeric {
    static func .* (_ lhs: Tensor, _ rhs: Tensor) -> Tensor {
        return Raw.mul(lhs, rhs)
    }
}

let x: Tensor<Double> = [[1.0, 2.0], [3.0, 4.0]]
let y: Tensor<Double> = [[8.0, 7.0], [6.0, 5.0]]
x .* y

## 래퍼 함수에서 파생된 것을 정의하기

당신은 쉽게 Swift API를 다듬어지지 않은 텐서플로우 연산자에서 정의할 수 있을 뿐만 아니라, Swift가 가지고 있는 최고 수준의 자동 미분계산 기능을 이용하여 다듬어지지 않은 텐서플로우 연산자를 미분할 수 있습니다.

`.*`연산자를 미분 가능하게 만들기 위해서, `@differentiable` 속성을 이용하세요. 그리고 `vjp:` 레이블에서 고안된 함수를 속성 인자로써 구체화 시키세요. `.*`연산자가 제네릭 타입의 `Scalar`가 `Numeric`를 따르도록 정의되었기 때문에, `Tensor<Scalar>`가 `Differentiable` 프로토콜을 따르도록 만들기엔 충분하지(적절하지) 않습니다. 타입 세이프티를 지키는, Swift는 우리에게 `@differentiable` 속성의 `Scalar`가  `TensorFlowFloatingPoint` 프로토콜을 따르도록 요구하기 위해서, 제네릭 제약 조건을 추가하도록 요구할 것입니다. `TensorFlowFloatingPoint`는 `Tensor<Scalar>`가 `Differentiable`을 따르도록 만드는 프로토콜 입니다.

```swift
@differentiable(vjp: multiplyDerivative where Scalar: TensorFlowFloatingPoint)
```

In [ ]:
infix operator .* : MultiplicationPrecedence

extension Tensor where Scalar: Numeric {
    @differentiable(vjp: multiplyDerivative where Scalar: TensorFlowFloatingPoint)
    static func .* (_ lhs: Tensor,  _ rhs: Tensor) -> Tensor {
        return Raw.mul(lhs, rhs)
    }
}

extension Tensor where Scalar : TensorFlowFloatingPoint { 
    static func multiplyDerivative(
        _ lhs: Tensor, _ rhs: Tensor
    ) -> (Tensor, (Tensor) -> (Tensor, Tensor)) {
        return (lhs * rhs, { v in
            ((rhs * v).unbroadcasted(to: lhs.shape),
            (lhs * v).unbroadcasted(to: rhs.shape))
        })
    }
}

// Now, we can take the derivative of a function that calls `.*` that we just defined.
gradient(at: x, y) { x, y in
    (x .* y).sum()
}

## 더 많은 예제들

In [ ]:
let matrix = Tensor<Float>([[1, 2], [3, 4]])

print(Raw.matMul(matrix, matrix, transposeA: true, transposeB: true))
print(Raw.matMul(matrix, matrix, transposeA: true, transposeB: false))
print(Raw.matMul(matrix, matrix, transposeA: false, transposeB: true))
print(Raw.matMul(matrix, matrix, transposeA: false, transposeB: false))